In [7]:
from tifffile import imread
import tifffile
import numpy as np
import os
import pandas as pd

In [8]:

input_dir = r"D:\A52b_DISCOVAIR_R3\Cycle 4\2022_03_30_07_51_38--588B LNG2_Cycle 4_001/"
input_dir = input_dir.replace("\\", "/")
input_dir = input_dir.replace("\\", "/")
output_dir = 'D:\Processing/588B_LNG2/'
files = os.listdir(input_dir)

In [9]:
tifs =  [k for k in files if 'dw' not in k] # filter for deconvolved images
tifs =  [k for k in tifs if '.tif' in k]
tifs =  [k for k in tifs if '.txt' not in k]

In [10]:
tifs =  [k for k in tifs if 'Corrected' in k]
split_underscore = pd.DataFrame(tifs)[0].str.split('--', expand = True)


In [11]:
bases = [4]
tiles = sorted(split_underscore[1].unique())
channels = split_underscore[3].unique()

In [12]:

folder_output = output_dir + '/preprocessing/2DTiles/'
if not os.path.exists(folder_output):
    os.makedirs(folder_output)
for ååå, w in enumerate(sorted(bases)):
    for f,i in enumerate(tiles):
        print(i)
        tifs_base_tile = [k for k in tifs if str(i)+'--' in k]
        file_name = (folder_output+'Base_'+ str(int(w)) + '_t' + str(int(i.split('Stage')[1])+1) + '.tif')
        if os.path.isfile(file_name)==True and os.path.getsize(file_name)>40000000:
            print('already exists')
        else:
            with tifffile.TiffWriter(file_name) as stack:
                for å,z in enumerate(sorted(list(channels))):
                    tifs_base_tile_channel = [k for k in tifs_base_tile if str(z) in k]
                    maxi = np.zeros((2048,2048))
                    for q in (tifs_base_tile_channel):
                        im_array = imread(input_dir + q)
                        inds = im_array > maxi # find where image intensity > max intensity
                        maxi[inds] = im_array[inds]
                    maxi = maxi.astype('uint16')
                    stack.save(maxi)

Stage00
Stage01
Stage02
Stage03
Stage04
Stage05
Stage06
Stage07
Stage08
Stage09
Stage10
Stage100
Stage101
Stage102
Stage103
Stage104
Stage105
Stage106
Stage107
Stage108
Stage109
Stage11
Stage110
Stage111
Stage112
Stage113
Stage114
Stage12
Stage13
Stage14
Stage15
Stage16
Stage17
Stage18
Stage19
Stage20
Stage21
Stage22
Stage23
Stage24
Stage25
Stage26
Stage27
Stage28
Stage29
Stage30
Stage31
Stage32
Stage33
Stage34
Stage35
Stage36
Stage37
Stage38
Stage39
Stage40
Stage41
Stage42
Stage43
Stage44
Stage45
Stage46
Stage47
Stage48
Stage49
Stage50
Stage51
Stage52
Stage53
Stage54
Stage55
Stage56
Stage57
Stage58
Stage59
Stage60
Stage61
Stage62
Stage63
Stage64
Stage65
Stage66
Stage67
Stage68
Stage69
Stage70
Stage71
Stage72
Stage73
Stage74
Stage75
Stage76
Stage77
Stage78
Stage79
Stage80
Stage81
Stage82
Stage83
Stage84
Stage85
Stage86
Stage87
Stage88
Stage89
Stage90
Stage91
Stage92
Stage93
Stage94
Stage95
Stage96
Stage97
Stage98
Stage99


In [13]:
import os
from xml.dom import minidom
path = input_dir +  '/metadata/' 
metadatafiles = [n for n in os.listdir(path) if os.path.isfile(os.path.join(path,n))]
metadatafiles_cleaned =  [k for k in metadatafiles if '.xlif' in k]
output_dir_int=output_dir + '/preprocessing/2DTiles/'
for p, meta in enumerate(metadatafiles_cleaned):
    for base in range(5):
            metadata_txt = open(output_dir_int +"TileConfiguration_Base_" + str(base+1) + ".txt","a") 
            metadata_txt.write('# Define the number of dimensions we are working on\n')
            metadata_txt.write('dim = 2\n\n# Define the image coordinates\n')
            mydoc = minidom.parse(path + meta)
            tile =[]
            x =[]
            y =[]
            items = mydoc.getElementsByTagName('Tile')
            for elem in items:

                tile.append(int(elem.attributes['FieldX'].value))
                x.append(float(elem.attributes['PosX'].value))
                y.append(float(elem.attributes['PosY'].value))


            dict = {'x': x, 'y': y}     
            df = pd.DataFrame(dict) 
            df['x'] =((df.x-np.min(df.x))/.000000321) + 1
            df['y'] =((df.y-np.min(df.y))/.000000321) + 1

            for elem in items:
                tile_value = elem.attributes['FieldX'].value
                x_value = ((float(elem.attributes['PosX'].value))-np.min(x))/.000000321 + 1
                y_value = ((float(elem.attributes['PosY'].value))-np.min(y))/.000000321 + 1
                metadata_txt.write('Base_' + str(base+1) + '_t' + tile_value +'; ; ' + '(' + str(x_value)+ ', ' + str(y_value) + ')' + '\n')
            df.to_csv(output_dir_int + 'tile_coordinates_Base_'+str(base+1)+'.csv', index = False, header = None)